# 11.2.5 Rank-k Approximation

<font color=red> Be sure to make a copy!!!! </font>

This notebook walks you through the operations required to compute a low rank approximation of a matrix $ B $. We will create a matrix $A$ whose column space will be used in the approximation of $B$.

We start by creating a random $ m \times n $ matrix $ B $.  We then take $ k $ columns of $ B $ to be matrix $ A $, whose columns will be used in the approximation $ B \approx A V $.
(In the text and the videos, we talk about computing $ W $ so that $ B \approx A W^T $.  Here we find it more convenient to compute the transpose of that matrix instead.  We call it $ V $ to distinguish it from $ W $.  So, $ W = V^T $.)

$ V $ is computed as $ ( A^T A )^{-1} A^T B $. 

In [5]:
include("../flame.jl")
include("../laff/laff.jl")

m = 8
n = 8
k = 3

# Random matrix of size mxn
B = rand( m, n )

# A is k columns of B taken at even intervals
if 2*k <= n #k is less than half of n
    interval = cld( n, k ) 
    A = B[ :, 1:interval:end ] # These are slices in Julia.
                               # This says take all rows of B, and columns 
                               # from 1 to the end at interval steps
else
    A = B[ :, 1:k] #If k is greater than half of n, then just take the first k columns
end

println("A = ")
A

A = 


8×3 Array{Float64,2}:
 0.343938  0.742081  0.421397
 0.619161  0.765847  0.197177
 0.762933  0.798185  0.890511
 0.213486  0.175195  0.962225
 0.302733  0.470232  0.664115
 0.703224  0.206922  0.431282
 0.383473  0.50021   0.31639 
 0.774145  0.501501  0.416034

In [6]:
println("B = ")
B

B = 


8×8 Array{Float64,2}:
 0.343938  0.28445   0.594996  0.742081  …  0.738566   0.421397  0.487216
 0.619161  0.769376  0.652634  0.765847     0.195886   0.197177  0.563184
 0.762933  0.258515  0.114408  0.798185     0.171687   0.890511  0.371801
 0.213486  0.752262  0.647766  0.175195     0.423964   0.962225  0.970565
 0.302733  0.257787  0.561444  0.470232     0.182338   0.664115  0.481812
 0.703224  0.592994  0.768478  0.206922  …  0.997846   0.431282  0.799457
 0.383473  0.21813   0.797617  0.50021      0.0807512  0.31639   0.984913
 0.774145  0.180572  0.917437  0.501501     0.24827    0.416034  0.808492

We start the process of computing $( A^T A )^{-1} A^T B$ by computing $ A^T A $ and storing the result in a matrix, $C$.  In this implementation, we ignore symmetry.

In [7]:
C = transpose( A ) * A 

println( "C = " )
C

C = 


3×3 Array{Float64,2}:
 2.46182  2.24369  2.09958
 2.24369  2.57065  2.11152
 2.09958  2.11152  2.83558

Now, form $ V = A^T B $, notice that we are not done forming $V$ after this step.

In [8]:
V = transpose( A ) * B

3×8 Array{Float64,2}:
 2.46182  1.65051  2.56078  2.24369  1.67658  1.57687  2.09958  2.71877
 2.24369  1.58203  2.42825  2.57065  1.93655  1.36653  2.11152  2.54978
 2.09958  1.79671  2.44293  2.11152  1.81098  1.59098  2.83558  2.8941 

Instead of computing $ C^{-1} = ( A^T A )^{-1} $ explicitly, we notice that we can instead store the $ L $ and $ U $ factorization of $C$ in $ C $ and then just solve $ L ( U X ) = V $. First we will overwrite $ V $ with the 
result of solving $ L Z = V $, and then we will overwrite $ V $ with the result of solving $ U X = V $.

Copy your `LU_unb_var5` routine from *Notebook 6.3: Solving A x b via LU Factorization and Triangular Solves*

In [9]:
include("../flame.jl")
include("../laff/laff.jl")

function LU_unb_var5!(A)

    ATL, ATR, 
    ABL, ABR  = flame.part_2x2(A, 
                               0, 0, "TL")

    while size(ATL, 1) < size(A, 1)

        A00,  a01,     A02,  
        a10t, alpha11, a12t, 
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, 
                                                       ABL, ABR, 
                                                       1, 1, "BR")

        #------------------------------------------------------------#

        laff.invscal!( alpha11, a21 )        #  a21 := a21 / alpha11
        laff.ger!( -1.0, a21, a12t, A22 )    #  A22 := A22 - a21 * a12t

        #------------------------------------------------------------#

        ATL, ATR, 
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  
                                               a10t, alpha11, a12t, 
                                               A20,  a21,     A22,  
                                               "TL")

    end

    flame.merge_2x2!(ATL, ATR, 
                     ABL, ABR, A)

end

LU_unb_var5! (generic function with 1 method)

Now run `LU_unb_var5` on the matrix $C$ to store $L$ and $U$ in it.

In [10]:
LU_unb_var5!( C )

Solve $L ( U X ) = V$, overwriting $V$ where $U$ and $L$ are stored in the upper and the strictly lower portions of $C$ respectively.

In [11]:
laff.trsm!("Lower triangular", "No transpose", "Unit diagonal", C, V)
laff.trsm!("Upper triangular", "No transpose", "Nonunit diagonal", C, V)

The $ j $th column of $ A V $ now equals the projection of the $ j $th column of $ B $ onto the column space of $ A $, $ {\cal C}( A ) $. 

A couple of notes:
    
-    The matrix $ A^T A $ is symmetric positive definite.  As a result, one does not need to pivot when performing LU factorization.  (The reason for this is beyond the scope of this course.)

-    One could use what is called a "Symmetric rank-k update" operation to compute only the lower (or upper) triangular part of $ A^T A $.  This would (approximately) halve the number of floating point operations that are required.

-    In one of the enrichments, 8.4.2, we discussed the Cholesky factorization of a symmetric positive definite matrix.   One should ideally use that here, since it also takes advantage of symmetry.

- This would then leave us with $ L $, a lower triangular matrix, such that $ C = A^T A = L L^T $.  Computing $ V $ would then require the steps
  - $ V = A^T B $.
  - Solve $ L Z = V $ overwriting $ V $ with $ Z $.
  - Solve $ L^T X = V $ overwriting $ V $ with $ X $.
    

## A routine

The above computation should be implemented as the routine <code> RankKApprox( B, k ) </code>
where $ B $ is the $ m \times n $ matrix to be approximated, and $k$ is the rank of the eventual approximation that will be returned by the method. 

In [12]:
function RankKApprox( B, k )
    m, n = size(B) # How many rows and columns does B have?

    # A is k columns of B taken at even intervals
    if 2*k <= n #k is less than half of n
        interval = cld( n, k ) 
        A = B[ :, 1:interval:end ] # These are slices in Julia.
                                   # This says take all rows of B, and columns 
                                   # from 1 to the end at interval steps
    else
        A = B[ :, 1:k] #If k is greater than half of n, then just take the first k columns
    end
    
    # C = A^T A
    C = transpose( A ) * A   
    # W = A^T B
    W = transpose( A ) * B
    # Overwrite C with its LU factorization
    LU_unb_var5!( C )
    
    # Solve L(UX) = W, overwriting W with X
    laff.trsm!("Lower triangular", "No transpose", "Unit diagonal", C, W)
    laff.trsm!("Upper triangular", "No transpose", "Nonunit diagonal", C, W)
    
    return A * W
end

RankKApprox (generic function with 1 method)

## An Application: Rank k Image Approximation

Now that we have implemented routines to create low rank approximations to matrices we will explore what a rank k approximation to an image looks like. Each pixel in an image can be thought of as a value in a matrix. For a grayscale image, this value corresponds to how black or white it is on a relative scale.

We will use two techniques for these approximations. First, the normal approximation developed above and second, the SVD which is a very useful matrix decomposition that guarantees the best approximation given $k$ columns. The SVD might take a while to compute, so don't panic if one of the code blocks takes a bit to complete.

Try experimenting with the number of columns below by changing the `numCols` variable.

If you want to use your own images, make sure that they are in the `png` format and just place them in your notebooks directoy. Then change the `filename` variable to reflect the file name of your image.

In [16]:
using Pkg; Pkg.add("Images")
using Image

# Try varying the number of columns used for the approximation 
numCols = 40

# Load an image (stored in the current working directory)
filename = "building.png"

img = load( filename )

 Resolving package versions...
 Installed IdentityRanges ──────────── v0.2.0
 Installed Images ──────────────────── v0.16.1
 Installed SpecialFunctions ────────── v0.7.2
 Installed ImageAxes ───────────────── v0.5.0
 Installed SIUnits ─────────────────── v0.1.0
 Installed ImageTransformations ────── v0.7.0
 Installed ComputationalResources ──── v0.3.0
 Installed ImageCore ───────────────── v0.7.3
 Installed WoodburyMatrices ────────── v0.4.1
 Installed MappedArrays ────────────── v0.2.1
 Installed SimpleTraits ────────────── v0.8.0
 Installed ImageShow ───────────────── v0.1.2
 Installed IterTools ───────────────── v1.1.1
 Installed AxisArrays ──────────────── v0.3.0
 Installed IndirectArrays ──────────── v0.5.0
 Installed Distances ───────────────── v0.7.3
 Installed ImageDistances ──────────── v0.1.1
 Installed Rotations ───────────────── v0.9.1
 Installed FFTW ────────────────────── v0.2.4
 Installed IntervalSets ────────────── v0.3.1
 Installed FFTViews ────────────────── v0.2.0
 I

ArgumentError: ArgumentError: Package Image not found in current path:
- Run `import Pkg; Pkg.add("Image")` to install the Image package.


In [ ]:
# Make the approximations using normal equations and the SVD
# This step might take a while as it is a lot of computation.
normalApprox, SVDApprox = create_approximations( img, k=numCols, approximator=RankKApprox )

In [ ]:
#Now plot the approximations that we have created

# Note that we're having some issues with our 
# approximations being somewhat darker than the 
# real image and are investigating.
plot_approximations( img, normalApprox, SVDApprox, k=numCols )

## Hints for implementing RankKApprox

<font color=blue> # C = A^T A: </font> This comment corresponds to computing $C = A^T A$. Try using numpy's built in transpose method by calling `np.transpse(A)`. 

<font color=blue># W = A^T B: </font> This comment corresponds to computing $W = A^T B$. See above for a hint.

<font color=blue># Overwrite C with its LU factorization: </font> Use your implementation of LU_unb_var5 from an earlier notebook for this.
    
<font color=blue># Solve L(UX) = W, overwriting W with X: </font> Use `laff.trsm` to do this. Recall that "trsm" means triangular solve with multiple right hand sides.